# Robyn Marketing Mix Modeling with Python SageMaker

This is sample code to run [Robyn](https://github.com/facebookexperimental/Robyn) Marketing Mix Model library from Python using SageMaker.

The main purpose of this sample code is showing how to run encapsulated Robyn docker image in isolated SageMaker runtime and pass data / hyperparameters from python.

You basically need to upload data to S3 and run SageMaker Robyn Image as remote procedure and get result in S3.

The main Robyn code is in `app/train.R`, but it has minimal change from [Robyn demo code](https://github.com/facebookexperimental/Robyn/blob/main/demo/demo.R), so you still need to modify R code to handle production workload.

Tested with SageMaker `2.117.0`.

## Setup

In [ ]:
import sagemaker
import boto3
print(sagemaker.__version__)

In [ ]:
boto3_session = boto3.session.Session()
sage_session = sagemaker.Session()

bucket = sage_session.default_bucket()
role_arn = sagemaker.get_execution_role()
account_id = sage_session.account_id()
region = boto3_session.region_name

## Build Docker Image

Build docker image to run the SageMaker. Only run this once.

Docker image will be built with CodeBuild using `sm-docker`.

You need appropriate role to run CodeBuild. Please check [blog](https://aws.amazon.com/jp/blogs/machine-learning/bringing-your-own-custom-container-image-to-amazon-sagemaker-studio-notebooks/) for required permissions.

In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
ecr_repository_training = "sagemaker-robyn"
tag_training = "latest"
model_name = f"{ecr_repository_training}:{tag_training}"
model_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository_training}:{tag_training}"

In [ ]:
!sm-docker build ./docker --repository $model_image_uri --compute-type BUILD_GENERAL1_MEDIUM

Alternatively, you may clone this repo and run docker build from your local computer or Cloud9.

In [ ]:
# !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
# !aws ecr describe-repositories --repository-names $ecr_repository_training > /dev/null || aws ecr create-repository --repository-name $ecr_repository_training
# !docker build -t {ecr_repository_training}:{tag_training} ./docker > /dev/null
# !docker tag {ecr_repository_training}:{tag_training} $model_image_uri
# !docker push $model_image_uri > /dev/null

## Upload Data to S3 and define training input

In [ ]:
s3_holiday = sage_session.upload_data('data/dt_prophet_holidays.RData')
s3_train = sage_session.upload_data('data/dt_simulated_weekly.csv')
print(s3_holiday)
print(s3_train)

In [ ]:
s3_holiday_input = sagemaker.inputs.TrainingInput(s3_holiday)
s3_train_input = sagemaker.inputs.TrainingInput(s3_train)

## Train Model

Run Robyn in SageMaker Runtime.

You can pass data in S3 and hyperparameters from python and get result from S3.

You may need to customize `app/train.R` to run production workload. It has minimal change from [Robyn demo code](https://github.com/facebookexperimental/Robyn/blob/main/demo/demo.R)

In [ ]:
import json
# Sample Hyperparameter passed to Robyn
hyperparameters = {
    "window_start": "2016-11-21",
    "window_end": "2018-08-20"
}

train_estimator = sagemaker.estimator.Estimator(model_image_uri,
                       role_arn, 
                       source_dir="app/",
                       entry_point="train.R",
                       instance_count=1,
                       instance_type='ml.m5.xlarge',
                       output_path=f"s3://{bucket}/training_output",
                       sagemaker_session=sage_session,
                       environment={ "SM_HPS": json.dumps(hyperparameters) },
                       hyperparameters=hyperparameters)
train_estimator.fit({
    "train": s3_train_input,
    "holiday": s3_holiday_input
})